# How to log classification table

In [1]:
import wandb
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import wandb

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    model.train()
    total_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        total_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return model, total_loss


def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct, test_loss

In [2]:
import numpy as np

PROJECT = "KOREA-basic-logging" 
CONFIG = {
    "learning_rate": 1e-3,
    "epochs": 30,
    "batch_size": 64
}
classes = [
        "T-shirt",
        "Trouser",
        "Pullover",
        "Dress",
        "Coat",
        "Sandal",
        "Shirt",
        "Sneaker",
        "Bag",
        "Ankle boot",
    ]


## Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

with wandb.init(project=PROJECT, name='classification_image_table') as run:
    wandb.config.update(CONFIG)
    
    train_dataloader = DataLoader(training_data, batch_size=CONFIG['batch_size'])
    test_dataloader = DataLoader(test_data, batch_size=CONFIG['batch_size'])
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = NeuralNetwork().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=CONFIG['learning_rate'])
    epochs = CONFIG['epochs']
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        model, total_loss = train(train_dataloader, model, loss_fn, optimizer, device)
        correct, test_loss = test(test_dataloader, model, loss_fn, device)
        wandb.log({"train_loss": total_loss / len(train_dataloader)}, step=t)
        wandb.log({"test_loss": test_loss, "test_acc": correct}, step=t)
        #####################################################################
        if t == 1:
            model.eval()
            table = wandb.Table(columns=["Image", "True", "Pred"]+["score_"+str(cls) for cls in classes])
            for i in range(10000):
                x, y = test_data[i][0], test_data[i][1]
                with torch.no_grad():
                    pred = model(x)
                    predicted, actual = classes[pred[0].argmax(0)], classes[y]
                    row = [wandb.Image(x), actual, predicted]
                for s in nn.functional.softmax(pred[0], dim=0):
                    row.append(np.round(s.item(), 4))
                table.add_data(*row)
            wandb.log({"classification_image_table": table})    
            model.train()
        #####################################################################

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hyunwoo-oh (apac-partners). Use `wandb login --relogin` to force relogin


Epoch 1
-------------------------------
loss: 2.309580  [    0/60000]
loss: 2.288000  [ 6400/60000]
loss: 2.269484  [12800/60000]
loss: 2.257367  [19200/60000]
loss: 2.252481  [25600/60000]
loss: 2.219344  [32000/60000]
loss: 2.222229  [38400/60000]
loss: 2.191731  [44800/60000]
loss: 2.192310  [51200/60000]
loss: 2.153373  [57600/60000]
Test Error: 
 Accuracy: 45.3%, Avg loss: 2.151586 

Epoch 2
-------------------------------
loss: 2.167287  [    0/60000]
loss: 2.152312  [ 6400/60000]
loss: 2.091537  [12800/60000]
loss: 2.103773  [19200/60000]
loss: 2.057420  [25600/60000]
loss: 1.994780  [32000/60000]
loss: 2.013555  [38400/60000]
loss: 1.938257  [44800/60000]
loss: 1.950415  [51200/60000]
loss: 1.865383  [57600/60000]
Test Error: 
 Accuracy: 57.6%, Avg loss: 1.869274 

Epoch 3
-------------------------------
loss: 1.909478  [    0/60000]
loss: 1.875733  [ 6400/60000]
loss: 1.751777  [12800/60000]
loss: 1.791301  [19200/60000]
loss: 1.679223  [25600/60000]
loss: 1.636403  [32000/600

test_acc,▁▃▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇███████████
test_loss,█▇▅▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▇▆▄▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_acc,0.8056
test_loss,0.55368
train_loss,0.53301


# How to log segmentation table

In [1]:
import wandb
import glob
import numpy as np
from PIL import Image
BDD_CLASSES = {i:c for i,c in enumerate(['background', 'road', 'traffic light', 'traffic sign', 'person', 'vehicle', 'bicycle'])}

def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    unique = list(np.unique(mask_data))
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def generate_fake_prediction(mask):
    from scipy.ndimage import gaussian_filter
    mask = gaussian_filter(mask, sigma=10)
    return mask

fns = [fn.split('/')[-1].split('.jpg')[0] for fn in glob.glob('./artifacts/bdd_simple_1k_split_v3:v3/images/*')]
image_path = './artifacts/bdd_simple_1k_split_v3:v3/images/%s.jpg'
label_path = './artifacts/bdd_simple_1k_split_v3:v3/labels/%s_mask.png'

In [2]:
with wandb.init(project='KOREA-basic-logging', name="segmentation_image_table", job_type='create_table') as run:
    labels = [str(BDD_CLASSES[_lab]) for _lab in list(BDD_CLASSES)]
    table = wandb.Table(columns=["File_Name", "Images", "Split"] + labels)
    for fn in fns[:10]:
        raw_image = Image.open(image_path%fn)
        mask_image = Image.open(label_path%fn)

        class_in_image = get_classes_per_image(np.array(mask_image), BDD_CLASSES)
        mask_data = np.array(mask_image)
        prediction_data = generate_fake_prediction(np.array(mask_image))
        
        table.add_data(
            str(fn),
            wandb.Image(
                    raw_image,
                    masks={
                        "ground_truths": {
                            "mask_data": mask_data,
                            "class_labels": BDD_CLASSES,
                        },
                        "predictions": {
                            "mask_data": prediction_data,
                            "class_labels": BDD_CLASSES,
                        }
                    },
            ),
            "None",
            *[class_in_image[_lab] for _lab in labels]
        )
    wandb.log({"segmentation_image_table": table})

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hyunwoo-oh (apac-partners). Use `wandb login --relogin` to force relogin


# How to log object-detection table

In [1]:
import wandb
import numpy as np
from PIL import Image
from pathlib import Path
from pycocotools.coco import COCO

# COCO 데이터셋 경로 설정
image_dir = Path('./coco/images/val2017')
annotation_file = './coco/annotations/instances_val2017.json'

# COCO 데이터셋 로드 및 클래스 매핑 생성
coco = COCO(str(annotation_file))

# COCO 클래스 정의를 API에서 가져오기
categories = coco.loadCats(coco.getCatIds())

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


In [2]:
COCO_CLASSES = {cat['id']: cat['name'] for cat in categories}

def get_boxes_for_image(image_id, image_size):
    annotations = coco.loadAnns(coco.getAnnIds(imgIds=[image_id]))
    boxes = []
    for ann in annotations:
        x, y, width, height = ann['bbox']
        x_min = x
        y_min = y
        x_max = min(x + width, image_size[0])  # x_max는 이미지의 너비를 넘지 않도록 설정
        y_max = min(y + height, image_size[1])  # y_max는 이미지의 높이를 넘지 않도록 설정
        boxes.append({
            "position": {"minX": x_min, "minY": y_min, "maxX": x_max, "maxY": y_max},
            "class_id": ann['category_id'],
            "box_caption": COCO_CLASSES[ann['category_id']],
            "scores": {"score": np.random.uniform(0.5, 1.0)},  # 랜덤 스코어로 가짜 예측을 생성
            "domain" : "pixel",
        })
    return boxes

In [3]:
with wandb.init(project='KOREA-basic-logging', name="object_detection_image_table", job_type='create_table') as run:
    # Weights & Biases 로깅 설정
    labels = [str(COCO_CLASSES[_lab]) for _lab in COCO_CLASSES]
    table = wandb.Table(columns=["File_Name", "Images"]+labels)

    # 이미지 10개만 선택하여 기록
    for image_id in coco.getImgIds()[:10]:
        image_info = coco.loadImgs(image_id)[0]
        image_path = image_dir / image_info['file_name']
        raw_image = Image.open(image_path)

        boxes = get_boxes_for_image(image_id, raw_image.size)

        score_in_image = {label: 0 for label in labels}
        for box in boxes:
            score_in_image[box["box_caption"]] = box["scores"]["score"]

        # 테이블에 데이터 추가
        table.add_data(
            str(image_info['file_name']),
            wandb.Image(
                raw_image,
                boxes={
                    "predictions": {
                        "box_data": boxes,
                        "class_labels": COCO_CLASSES,
                    }
                },
            ),
            *[score_in_image[_lab] for _lab in labels]
        )
    wandb.log({"object_detection_image_table": table})

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hyunwoo-oh (apac-partners). Use `wandb login --relogin` to force relogin
